# Диагностика модели: регрессия с серийной корреляцией

In [ ]:
import pandas as pd
import numpy as np

import statsmodels.formula.api as smf
from statsmodels.stats.api import durbin_watson, acorr_breusch_godfrey, cov_hac # DW & LM-тесты
from statsmodels.iolib.summary2 import summary_col # вывод подгонки

from scipy.stats import chi2 # chi2-распределение и критические значения

# настройки визуализация
import matplotlib.pyplot as plt

# Не показывать Warning
import warnings
warnings.simplefilter(action='ignore', category=Warning)

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

<i><b><span style="color: purple">Диагностика модели: тесты на серийную корреляцию</span> </b><br>

Рассмотрим __линейную регрессию для временных рядов__ с серийной корреляцией для ошибки (внешнего шока)

$$
\begin{aligned}
	y_t&=x_t^\top\beta+u_t \\ u_t&=\rho_1u_{t-1}+\cdots+\rho_pu_{t-p}+v_t
\end{aligned}
$$
где
* $y_t,x_t$ наблюдаемы
* $u_t$ - эндогенный внешний шок (__ненаблюдаем!__)
* $v_t$ - экзогенный внешний шок (__ненаблюдаем!__)
* $p$ - порядок серийной корреляции/автокорреляции

**<span style="color:purple">DW-тест:</span>** тестируем автокорреляцию __первого порядка__, т.е. для модели

$$
\begin{aligned}
	y_t&=x_t^\top\beta+u_t \\ u_t&=\rho u_{t-1}+v_t
\end{aligned}
$$
тестируем

$$
\begin{aligned}
	H_0:\rho&=0 & &vs & H_1:\rho&\ne0
\end{aligned}
$$

__Тестовая статистика__

$$
\begin{aligned}
	DW&=\frac{\sum_{t=2}^n(e_t-e_{t-1})^2}{\sum_{t=1}^n e^2_t} & 0\leq DW&\leq4
\end{aligned}
$$

__Критические значения__: из специальной таблицы находим два критических значения $0<d_\ell<d_u$ (зависят от $n,k,\alpha$)

__Вывод:__ 

|Значение $DW$ |Гипотеза $H_0$|Вывод |
|:--:|:--:|:--:|
|$0 \leq DW< d_L$|отвергается|есть положительная автокорреляция|
|$d_L \leq DW \leq d_U$ |?|неопределенность |
|$d_U < DW < 4-d_U$ |не отвергается |автокорреляция отсутствует |
|$4-d_U \leq DW \leq 4-d_L$|?|неопределенность |
|$4-d_L < DW \leq 4$ |отвергается|есть отрицательная автокорреляция|

__Вывод (приближённый):__
* Не отвергаем гипотезу при $DW\approx2$ (нет серийной корреляции)
* Отвергаем гипотезу если $DW$ "сильно отличается от 2" (есть серийная корреляция)

__Замечание:__ 

* используем метод [`statsmodels.stats.stattools.durbin_watson(resids)`](https://www.statsmodels.org/stable/generated/statsmodels.stats.stattools.durbin_watson.html#statsmodels.stats.stattools.durbin_watson)
* где `resids` -  остатки модели
* критические значения берём из специальной таблицы
 

**<span style="color:purple">LM/BG-тест:</span>** тест на  серийную корреляцию произвольного порядка

$$
	H_0:\rho_1=\cdots=\rho_p=0
$$

Тест основан на вспомогательной (неинтерпретируемой!) регрессии для OLS-остатков

$$
\begin{aligned}
	e_t\;&\text{на}\; e_{t-1},\ldots,e_{t-p}, x & &R^2_{aux}
\end{aligned}
$$

__Два подхода к тестированию__
* тестовая статистика $LM=(n-p)R^2_{aux}$, критическое значение $\chi^2_{cr}=\chi^2_{df=p}(\alpha)$ ($p$ - порядок серийной корреляции)
* F-тест на совместную значимость $e_{t-1},\ldots,e_{t-p}$

__Вывод:__ Тест указывает на серийную корреляцию (отвергаем $H_0$) при $LM>\chi^2_{cr}, F>F_{cr},P<\alpha$. 

__Замечание:__ 

* используем метод [`statsmodels.stats.diagnostic.acorr_breusch_godfrey(res, nlags)`](https://www.statsmodels.org/stable/generated/statsmodels.stats.diagnostic.acorr_breusch_godfrey.html#statsmodels.stats.diagnostic.acorr_breusch_godfrey)
* где `res` -  подогнанная модель
* `nlags` – порядок автокорреляции
* методы возвращает четыре числа: $LM$-статистику и её P-значение, $F$-статистику и её P-значение

</div>

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

<i><b><span style="color: purple">Эконометрика регрессии с серийной корреляцией</span> </b><br>

Рассмотрим __линейную регрессию для временных рядов__ с серийной корреляцией для ошибки (внешнего шока)

$$
\begin{aligned}
	y_t&=x_t^\top\beta+u_t \\ u_t&=\rho_1u_{t-1}+\cdots+\rho_pu_{t-p}+v_t
\end{aligned}
$$
где
* $y_t,x_t$ наблюдаемы
* $u_t$ - эндогенный внешний шок (__ненаблюдаем!__)
* $v_t$ - экзогенный внешний шок (__ненаблюдаем!__)
* $p$ - порядок серийной корреляции/автокорреляции

__Цель__:
* оценить коэффициенты $\beta$
* тестировать гипотезы о коэффициентах (t-/F-тесты)

**<span style="color:purple">Подгонка модели:</span>** обычный OLS

**<span style="color:purple">Тестирование гипотез:</span>** необходимо использовать робастную оценку ковариационной матрицы коэффициентов.
__Для регрессии с серийной корреляцией используем HAC-оценку__ (HAC = Heteroskedasticity Autocorrelation Consistent) или оценку Newey-West.

При подгонке модели используем следующие настройки `.fit(cov_type='HAC', cov_kwds={'maxlags': None, 'use_correction':True})`

В `cov_kwds` в виде словаря задаём настройки ковариационной матрицы
* `'maxlags':None` означает, что число лагов будет выбрано равным $[4\sqrt[9]{(n/100)^2}]$ (Schwert rule, квадратные скобки означают целую часть числа)
* `'use_correction':True` означает, что будет учитываться поправка на малые выборки

</div>

Для датасета `Mishkin` рассмотрим регрессию

$$
	\Delta(pai3)\;\text{на}\;\Delta(tb3),\Delta(\log(cpi))
$$

где 
* $\Delta x_t = x_t-x_{t-1}$: первая разность или __приращение__ (период к периоду)
* $\Delta \log(x_t) = \log x_t-\log x_{t-1}$: лог-доходность или __процентное приращение__ (период к периоду)

In [ ]:
# загрузим данные
df = pd.read_csv('Mishkin.csv')

In [ ]:
# т.к. месячные данные, то зададим периодический временной индекс с 1950-02 по 1990-12
# параметр freq задаёт периодичность
df.index = pd.period_range(start='1950-02', end='1990-12', freq='M')
df

## Спецификация и подгонка

__Замечание__: сначала подготовим датасет, т.е. выполним необходимые преобразования данных и добавим новые переменные:
* дифференцируем `pai3, tb3` (новые переменные `diff_pai3, diff_tb3`)
* log-дифференцируем `cpi` (новая переменная `diff_log_cpi`)

In [ ]:
# добавим новые переменные в датасет:
df[ ['diff_pai3', 'diff_tb3'] ] = df[ ['pai3', 'tb3'] ].diff()
df['diff_log_cpi'] = np.log(df['cpi']).diff()
df

In [ ]:
# спецификация модели через формулу
mod = smf.ols(formula='diff_pai3~1+diff_tb3+diff_log_cpi', data=df)
# подгонка модели с ковариационной матрицей по умолчанию (неробастной)
res = mod.fit()

## DW-тест

Тестируем модель на серийную корреляцию 1-го порядка

In [ ]:
# DW-статистика
durbin_watson(res.resid)

### Вывод

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

Отвергаем гипотезу $H_0$, т.к. $DW$ "сильно отличается от 2", <i><b><span style="color: purple">есть серийная корреляция</span></b><br>

</div>

## LM/BG-тест

Тестируем модель на серийную корреляцию 2-го порядка 

In [ ]:
# LM-тест на автокорреляцию 2-го порядка
acorr_order = 2
lm, lmpval, fval, fpval = acorr_breusch_godfrey(res, nlags=acorr_order)
lm, lmpval

## Критическое значение $\chi^2$-распределения

In [ ]:
# Задаём уровень значимости
sign_level = 0.05
# Критическое значение распределения chi2
chi2.ppf(q=1-sign_level, df=acorr_order)

### Вывод

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

Отвергаем $H_0$ (т.к. $LM>\chi^2_{cr}$), <i><b><span style="color: purple">тест указывает на серийную корреляцию</span></b><br>

</div>

## Робастный t-тест (HAC s.e.)

Используем HAC-оценку ковариационной матрицы

In [ ]:
# спецификация модели через формулу
mod = smf.ols(formula='diff_pai3~1+diff_tb3+diff_log_cpi', data=df)
# подгонка модели с робастной HAC-оценкой ковариационной матрицы
res_hac = mod.fit(cov_type='HAC', cov_kwds={'maxlags': None, 'use_correction':True})

### Результаты двух оцениваний в одной таблице

__Обратим внимание на коды значимости (количество звёздочек у коэффициентов)__

In [ ]:
summary_col(results=[res, res_hac], model_names=['OLS-CME', 'HAC-CME'], stars=True)